In [47]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

# Experimento 5: 
- Nuevas relacion entre variables
- Eliminamos variables no importantes del exp 04, nos quedamos que las menos.
- Kaggle =  0.391


##### Levantamos el dataset con target ya calculado

In [48]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

In [49]:
columnas_baseline = df.columns.tolist()
columnas_baseline

['product_id',
 'periodo',
 'nacimiento_producto',
 'muerte_producto',
 'mes_n',
 'total_meses',
 'producto_nuevo',
 'ciclo_de_vida_inicial',
 'cat1',
 'cat2',
 'cat3',
 'brand',
 'sku_size',
 'stock_final',
 'tn',
 'plan_precios_cuidados',
 'cust_request_qty',
 'cust_request_tn',
 'target']

##### Preprocesamiento a la minima expresión :)

In [50]:
# ##### aplicamos OHE
# df = preprocesamiento.aplicarOHE(df)
# df.shape

### Feature Engineering

##### Neural Prophet

In [51]:
neural_prophet_fe = pd.read_csv("./datasets/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 22)

##### Prophet

In [52]:
prophet_fe = pd.read_csv("./datasets/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 28)

##### FE Moviles

In [53]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
df = feature_engineering.get_rolling_medians(df, "tn", 201912)
df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 590)

In [54]:
df = feature_engineering.get_lags(df, "cust_request_qty", 201912)
df = feature_engineering.get_delta_lags(df, "cust_request_qty", 24)
df = feature_engineering.get_rolling_means(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_stds(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_mins(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_maxs(df, "cust_request_qty", 201912)

In [55]:
df = feature_engineering.get_lags(df, "stock_final", 201912)
df = feature_engineering.get_delta_lags(df, "stock_final", 24)
df = feature_engineering.get_rolling_means(df, "stock_final", 201912)
df = feature_engineering.get_rolling_stds(df, "stock_final", 201912)
df = feature_engineering.get_rolling_mins(df, "stock_final", 201912)
df = feature_engineering.get_rolling_maxs(df, "stock_final", 201912)
df.shape

(31362, 1500)

##### FE Moviles sobre otras variables

In [56]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

#  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty')

##### FE Calendario

In [57]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 1525)

##### FE sobre FE

In [58]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 1554)

##### Variables Exogenas

In [59]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 1557)

##### Nuevas FE

In [60]:
df = feature_engineering.create_ratio_features(df)
df = feature_engineering.enhance_lifecycle_features(df)
df = feature_engineering.create_category_features(df)
df = feature_engineering.create_regime_features(df)
df = feature_engineering.create_nonlinear_trends(df)
df = feature_engineering.create_temporal_interactions(df)
df = feature_engineering.create_asymmetric_window_features(df)
df = feature_engineering.recomendaciones_deepseek(df)
df = feature_engineering.get_nuevas_features(df)
df.shape

(31362, 1599)

##### Elimino aquellas que no sirven

In [61]:
importantes = pd.read_csv("./feature_importance/exp04_3.csv", sep=',', encoding='utf-8')
no_importantes = importantes[importantes['importance'] <= 100]
no_importantes = no_importantes[~no_importantes['feature'].isin(columnas_baseline)]
no_importantes

,feature,importance
67,tn_vs_prev_year,100
68,tn_delta_lag5_lag8,99
69,tn_rolling_min_4,96
70,tn_lag_5,95
71,tn_rolling_std_6,95
...,...,...
677,dia_del_year,0
678,cat2_TE,0
680,cat2_PIEL1,0
681,cat2_OTROS,0


In [62]:
cols_a_eliminar = no_importantes.feature.unique()
print(f"Antes de eliminar: {df.shape[1]} columnas")
df = df.drop(columns=cols_a_eliminar, errors='ignore')
print(f"Después de eliminar: {df.shape[1]} columnas")

Antes de eliminar: 1599 columnas
Después de eliminar: 1139 columnas


Eliminar object/categorical columnas

In [68]:
df = df.select_dtypes(exclude=['datetime', 'datetime64', 'object'])


##### EXP 1

-   


In [69]:
study, models, results = model_lgb.main_entrenar(df, VERSION="v13")

[I 2025-06-27 19:35:57,861] A new study created in memory with name: no-name-11f77fbc-b8ec-4e21-9340-5deed7cf2c21


Features: ['product_id', 'periodo', 'nacimiento_producto', 'muerte_producto', 'mes_n', 'total_meses', 'producto_nuevo', 'ciclo_de_vida_inicial', 'sku_size', 'stock_final', 'tn', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'trend', 'trend_add', 'yearly_add', 'trend_mult', 'yearly_mult', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3', 'tn_lag_4', 'tn_lag_6', 'tn_delta_lag1_lag2', 'tn_delta_lag1_lag3', 'tn_delta_lag1_lag4', 'tn_delta_lag1_lag5', 'tn_delta_lag1_lag6', 'tn_delta_lag1_lag7', 'tn_delta_lag2_lag3', 'tn_delta_lag2_lag4', 'tn_delta_lag2_lag5', 'tn_delta_lag2_lag6', 'tn_delta_lag2_lag7', 'tn_delta_lag3_lag4', 'tn_delta_lag3_lag5', 'tn_delta_lag3_lag6', 'tn_delta_lag3_lag7', 'tn_delta_lag4_lag5', 'tn_delta_lag4_lag6', 'tn_delta_lag4_lag7', 'tn_delta_lag5_lag6', 'tn_delta_lag6_lag7', 'tn_delta_lag6_lag8', 'tn_delta_lag7_lag8', 'tn_delta_lag8_lag9', 'tn_rolling_mean_2', 'tn_rolling_std_2', 'tn_rolling_std_3', 'tn_rolling_std_4', 'tn_rolling_std_5', 'tn_rolling_min_2', 'tn_ro

[I 2025-06-27 19:39:28,121] Trial 0 finished with value: 16.73293405913477 and parameters: {'max_depth': 9, 'learning_rate': 0.2536999076681772, 'n_estimators': 759, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352}. Best is trial 0 with value: 16.73293405913477.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[37]	valid_0's l1: 29.2289
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[21]	valid_0's l1: 31.0257
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[24]	valid_0's l1: 26.0436
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[56]	valid_0's l1: 14.9918
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[52]	valid_0's l1: 14.7185
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[50]	valid_0's l1: 15.0563
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[47]	valid_0's l1: 13.3216
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[118]	valid_0's l1: 13.5044
Tr

[I 2025-06-27 19:41:16,340] Trial 1 finished with value: 16.614086362831785 and parameters: {'max_depth': 13, 'learning_rate': 0.11114989443094977, 'n_estimators': 118, 'min_child_weight': 10, 'subsample': 0.9162213204002109, 'colsample_bytree': 0.6061695553391381, 'reg_alpha': 0.18182496720710062, 'reg_lambda': 0.18340450985343382}. Best is trial 1 with value: 16.614086362831785.


...

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[99]	valid_0's l1: 32.3611
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[128]	valid_0's l1: 29.6835
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[109]	valid_0's l1: 27.0951
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[187]	valid_0's l1: 14.3063
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[246]	valid_0's l1: 14.6808
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[212]	valid_0's l1: 14.5247
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[288]	valid_0's l1: 13.2084
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[333]	valid_0's l1: 11.9861
Training 

[I 2025-06-27 22:49:57,075] Trial 48 finished with value: 16.80229775331391 and parameters: {'max_depth': 19, 'learning_rate': 0.027564917022197495, 'n_estimators': 929, 'min_child_weight': 10, 'subsample': 0.7680620685019279, 'colsample_bytree': 0.6378550428386887, 'reg_alpha': 0.7472348181360324, 'reg_lambda': 0.8956130277869783}. Best is trial 41 with value: 12.389729267352534.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[135]	valid_0's l1: 18.7801
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[134]	valid_0's l1: 19.0898
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[137]	valid_0's l1: 18.5055
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[188]	valid_0's l1: 12.9634
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[230]	valid_0's l1: 13.4158
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[210]	valid_0's l1: 13.3035
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[289]	valid_0's l1: 10.765
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[351]	valid_0's l1: 11.2066
Training 

[I 2025-06-27 22:54:27,045] Trial 49 finished with value: 13.010816339336145 and parameters: {'max_depth': 16, 'learning_rate': 0.03574200182162175, 'n_estimators': 765, 'min_child_weight': 2, 'subsample': 0.9158690424805265, 'colsample_bytree': 0.5817147621039089, 'reg_alpha': 0.9951838734046929, 'reg_lambda': 0.9513861357843706}. Best is trial 41 with value: 12.389729267352534.



🏆 Mejores parámetros:
max_depth: 20
learning_rate: 0.030897880551777897
n_estimators: 544
min_child_weight: 1
subsample: 0.874619568382676
colsample_bytree: 0.6778820699438167
reg_alpha: 0.49149655286672733
reg_lambda: 0.7981828538456203
boosting_type: gbdt
objective: regression
metric: mae
verbosity: -1
deterministic: True
force_col_wise: True

Entrenando modelo final...
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[543]	valid_0's l1: 4.9737
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[544]	valid_0's l1: 5.03021
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[544]	valid_0's l1: 5.03189
Ensemble MAE Val: 4.9567

🎯 MAE Test Ensemble: 4.8619

Top 5 mejores trials:
1. MAE: 12.3897
2. MAE: 12.4282
3. MAE: 12.5026
4. MAE: 12.5130
5. MAE: 12.5292


In [ ]:
study, models, 

{'mae_val': 4.956717124092388,
 'mae_test': 4.8619254927456295,
 'ensemble_pred': array([ 1.40075789e+03,  1.42303071e+03,  9.47133785e+02, ...,
        -2.04589048e+00, -1.14853019e+00,  1.76687987e+00]),
 'best_params': {'max_depth': 20,
  'learning_rate': 0.030897880551777897,
  'n_estimators': 544,
  'min_child_weight': 1,
  'subsample': 0.874619568382676,
  'colsample_bytree': 0.6778820699438167,
  'reg_alpha': 0.49149655286672733,
  'reg_lambda': 0.7981828538456203,
  'boosting_type': 'gbdt',
  'objective': 'regression',
  'metric': 'mae',
  'verbosity': -1,
  'deterministic': True,
  'force_col_wise': True}}

In [72]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Supongamos que querés predecir para los períodos 202011 y 202012
periodos_future = [201912]
df_future = df[df['periodo'].isin(periodos_future)].copy()

# Asegurate de tener las mismas columnas que en entrenamiento
features = [col for col in df.columns if col not in ['target']]

# Predecir
future_preds = model_lgb.predict_with_ensemble(models, df_future, features)

# Si querés evaluar el MAE (solo si tenés target en esos períodos)
if 'target' in df_future:
    mae_future = mean_absolute_error(df_future['target'], future_preds)
    print(f"📊 MAE Future: {mae_future:.4f}")

# Guardar las predicciones
df_future['pred_ensemble'] = future_preds

📊 MAE Future: 27.5312


In [73]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df_future = df_future[df_future['periodo'] == 201912]
df_future = df_future[df_future['product_id'].isin(productos_ok['product_id'].unique())]
df_future = df_future[['product_id', 'pred_ensemble']]
df_future.rename(columns={'pred_ensemble': 'tn'}, inplace=True)
df_future.to_csv("./outputs/predicciones_exp_05_lgb_fe_v1.csv", index=False, sep=',')


In [74]:
df_future[df_future['tn'] < 0]

,product_id,tn
30745,20302,-5.153057
30747,20304,-3.151204
30811,20378,-0.119861
30812,20379,-0.674136
30815,20382,-2.807060
...,...,...
31353,21259,-3.817922
31354,21262,-4.824055
31355,21263,-4.935911
31358,21266,-4.755991


In [75]:
promedios = model_lgb.promedio_12_meses_780p()
df_future = df_future.merge(promedios, on='product_id', how='left')
df_future


,product_id,tn_x,tn_y
0,20001,1544.925094,1454.732720
1,20002,1121.113872,1175.437142
2,20003,882.853085,784.976407
3,20004,716.819294,627.215328
4,20005,748.605352,668.270104
...,...,...,...
775,21263,-4.935911,0.029993
776,21265,0.723631,0.089541
777,21266,-4.755991,0.094659
778,21267,-5.517247,0.092835


In [76]:
df_future.rename(columns={'tn_x': 'tn', 'tn_y': 'tn_12m'}, inplace=True)
df_future['tn'] = df_future['tn'].mask(df_future['tn'] < 0, df_future['tn_12m'])
df_future

,product_id,tn,tn_12m
0,20001,1544.925094,1454.732720
1,20002,1121.113872,1175.437142
2,20003,882.853085,784.976407
3,20004,716.819294,627.215328
4,20005,748.605352,668.270104
...,...,...,...
775,21263,0.029993,0.029993
776,21265,0.723631,0.089541
777,21266,0.094659,0.094659
778,21267,0.092835,0.092835


In [ ]:
df_future.drop(columns=['tn_12m'], inplace=True)
df_future.to_csv("./outputs/predicciones_exp_05_lgb_fe_v2.csv", index=False, sep=',')

Feature importance

In [78]:
model_lgb.feature_importance(df, models, "05")

In [79]:
importances = model_lgb.feature_importance_promedio(models, df.drop(columns=['target']))
importances.sort_values(by='importance', ascending=False, inplace=True) 
importances

,feature,importance
16,yearly_add,0.025628
0,product_id,0.017944
1084,tn_seasonal_naive,0.014613
1119,naive_forecast_error,0.014388
1080,tn_ytd_sum,0.012998
...,...,...
1053,stock_final_rolling_max_18,0.000000
1052,stock_final_rolling_max_17,0.000000
1051,stock_final_rolling_max_16,0.000000
1050,stock_final_rolling_max_15,0.000000
